In [2]:
import pandas as pd

# Load Data
users = pd.read_csv('../data/users.csv')
payments = pd.read_csv('../data/payments.csv')
logins = pd.read_csv('../data/logins.csv')

users['signup_date'] = pd.to_datetime(users['signup_date'])
payments['payment_time'] = pd.to_datetime(payments['payment_time'])
logins['login_time'] = pd.to_datetime(logins['login_time'])

# Rule 1: Large Payments Soon After Signup

merged_pay = pd.merge(users, payments, on='user_id')
merged_pay['delay'] = merged_pay['payment_time'] - merged_pay['signup_date']

fast_high_pay = merged_pay[(merged_pay['delay'].dt.total_seconds() < 600) & (merged_pay['amount'] > 500)]
print("⚠️ Users with fast large payments:")
print(fast_high_pay[['user_id', 'amount', 'delay']])

# Rule 2: Shared IP Addresses Across Accounts

ip_counts = logins.groupby('ip_address')['user_id'].nunique().reset_index()
ip_counts.columns = ['ip_address', 'num_users']

# Filter IPs shared by more than 1 user
shared_ips_df = ip_counts[ip_counts['num_users'] > 1]

print("\n⚠️ Shared IPs used by multiple users:")
print(shared_ips_df)
# Rule 3: >3 Devices Used in a Single Day

logins['date'] = logins['login_time'].dt.date

# Count unique devices per user per day
device_counts = (
    logins.groupby(['user_id', 'date'])['device']
    .nunique()
    .reset_index(name='device_count')  # This makes device_counts a proper DataFrame
)

# Filter: more than 3 devices in one day
suspicious_device_users = device_counts[device_counts['device_count'] > 3]

print("\n⚠️ Users with more than 3 devices in a day:")
print(suspicious_device_users)


⚠️ Users with fast large payments:
   user_id  amount           delay
0        1     999 0 days 00:08:00
1        2    1500 0 days 00:05:00

⚠️ Shared IPs used by multiple users:
    ip_address  num_users
4  80.100.24.1          2

⚠️ Users with more than 3 devices in a day:
   user_id        date  device_count
0        1  2023-01-05             4
